In [1]:
import pandas as pd
import numpy as np


In [3]:
from google.colab import files
uploaded = files.upload()

Saving tweets_flagged_v2.csv to tweets_flagged_v2.csv


In [4]:
import pandas as pd
data_df = pd.read_csv("tweets_flagged_v2.csv")
data_df = data_df.drop(columns = ["Unnamed: 0"])

In [5]:
data_df["Toxicity"].value_counts()

,count
Toxicity,
0,32592
1,24153


In [6]:
data_df.head(5)

,Toxicity,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [7]:
data_df.shape

(56745, 2)

In [8]:
data_df["Toxicity"].value_counts()

,count
Toxicity,
0,32592
1,24153


In [9]:
for _ in range(10):
    random_ind = np.random.randint(0, len(data_df))
    random_data = data_df.iloc[random_ind]
    print(random_data["tweet"], random_data["Toxicity"])

I'm fucking around with two bitches but i never made those hoes my mistress 1
@808NoHeartBreak hoes 1
"@datFullerkid: @TreVaughnLG yall both gimped up??" Ain't tht a bitch huh lol 1
 @user love will always win.â¤ï¸ð #pride #solidarity #equality #husbands #loveislove #newlyweds   @user  0
&#8220;@_BeautifulKeezy: I'm about too put sooooo much nut in my fucking mouth and gts frfr &#128514;&#128514;&#128514;&#128128;&#8221; bitch yu nasty 1
dress shopping with the bride-to-be today @user !! ð°ð½â¤ï¸   #keepittogethercait #bestfriends 0
Like bitch you're a man. That means You don't show fear or pain. You be strong when others can't. 1
RT @ThinkLikeRandy: When you've got 5 or 6 hoes every one of them sees her man as god.That's why you don't have sex with them very often. -&#8230; 1
have a fun weekend planned with mama next week   #girltime ððð 0
RT @HEGS_com: What's that bright yellow thing the ref just showed to Martins Indi?! I can't remember seeing one of those before

In [10]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
X = data_df["tweet"].values
y = data_df["Toxicity"].values

In [12]:
sequences = [sequence for sequence in X]
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [13]:
model_inputs

{'input_ids': <tf.Tensor: shape=(56745, 481), dtype=int32, numpy=
array([[  101,  1030,  5310, ...,     0,     0,     0],
       [  101,  1030,  5310, ...,     0,     0,     0],
       [  101, 12170, 14945, ...,     0,     0,     0],
       ...,
       [  101,  2402, 10131, ...,     0,     0,     0],
       [  101,  2017,  2226, ...,     0,     0,     0],
       [  101,  1066,  1066, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(56745, 481), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(56745, 481), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype

In [14]:
import tensorflow as tf


In [15]:
dataset = tf.data.Dataset.from_tensor_slices((model_inputs['input_ids'],y))

In [16]:
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [17]:
# training pipeline
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

In [19]:
model = Sequential(name="text-classifier")
model.add(Embedding(len(tokenizer.get_vocab()), 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))

model.add(Dense(128, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))

model.add(Dense(1, activation = 'sigmoid'))

In [20]:
model.summary()

Model: "text-classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(loss = "binary_crossentropy", optimizer = "Adam")

In [22]:
with tf.device("/device:GPU:0"):  # ensures GPU usage if available
    history = model.fit(
        train,                   # your training data
        epochs=1,                # number of passes through dataset
        batch_size=16,           # how many samples per gradient update
        validation_data=val      # validation data tuple
    )

2482/2482 ━━━━━━━━━━━━━━━━━━━━ 901s 360ms/step - loss: 0.2707 - val_loss: 0.1338


In [23]:
from tensorflow.keras.metrics import Precision, Recall, Accuracy
import numpy as np


pre = Precision()
rec = Recall()
acc = Accuracy()


for batch in test.as_numpy_iterator():
    x_true, y_true = batch

    y_hat = model.predict(x_true)
    y_hat = y_hat.squeeze()

    pre.update_state(y_true, y_hat)
    rec.update_state(y_true, y_hat)
    acc.update_state(y_true, y_hat)

# Print final results
print("Precision:", pre.result().numpy())
print("Recall:", rec.result().numpy())
print("Accuracy:", acc.result().numpy())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step


In [ ]:
import numpy as np


y_true = np.concatenate([y for x, y in test.as_numpy_iterator()])
y_hat = model.predict(test)


prob_accuracy = 1 - np.mean(np.abs(y_true - y_hat))

print("Probability-based accuracy:", prob_accuracy)


354/354 ━━━━━━━━━━━━━━━━━━━━ 22s 61ms/step
Probability-based accuracy: 0.5130471293129
